In [1]:
from parser import *

import itertools
import random
import pickle
import numpy as np
from operator import itemgetter
from collections import Counter
from itertools import izip
import scipy
import scipy.spatial.distance
from numpy.linalg import svd
from sklearn.cluster import AffinityPropagation
from collections import defaultdict
from sklearn.feature_selection import RFE
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectFpr, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

In [2]:
from summanlp_textrank import commons, graph, keywords, pagerank_weighted, \
                  summarizer, textrank, textcleaner, textrank_runtime_error

In [3]:
inputXML = "sample_1"
# inputXML = "sample_rawXML"
summaryFolder = "sample_summary"
bodyFolder = "sample_body"

In [4]:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# PARSE:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
documents, labels = parse_xml_folder(inputXML, summaryFolder, bodyFolder, writeOption='none')
surfaceFeatures = process_data(documents)

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
STAGE [1] -- PARSING XML -- from sample_1 ...
STAGE [2] -- PROCESSING DATA -- (tokenizing/tagging/stopwords/extracting) ...


In [5]:
def featurize(documents, surfaceFeatures):
    print "STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ..."
    features = []
    for docIndex, doc in enumerate(documents):
        documentFeatures = extract_document_wide_features(doc)
        documentFeatures.append(surfaceFeatures[docIndex])
        features += [ counter_sum(fl) for fl in izip(*documentFeatures) ]
    return features

In [6]:
def extract_document_wide_features(document):
    documentFeatures = []

    documentFeatures.append(textrank_keyphrase(document))
    documentFeatures.append(lexrank_keyphrase(document))
    documentFeatures.append(textrank_keyword(document))

    return documentFeatures

In [7]:
def counter_sum(counterTuple):
    counterSum = Counter()
    for ele in counterTuple:
        counterSum += ele
    return counterSum

In [8]:
def textrank_keyphrase(text):

    # Creates the graph and calculates the similarity coefficient for every pair of nodes.
    graph = commons.build_graph([ syntacticUnit for syntacticUnit in text])
    summarizer._set_graph_edge_weights(graph)
    # Remove all nodes with all edges weights equal to zero.
    commons.remove_unreachable_nodes(graph)

    # Ranks the tokens using the PageRank algorithm. Returns dict of sentence -> score
    pagerank_scores = summarizer._pagerank(graph)

    # Adds the summa scores to the sentence objects.
    # summarizer._add_scores_to_sentences(sentences, pagerank_scores)

    results = []
    for su in text:
        score = (1-pagerank_scores[su.label, su.index]) if (su.label, su.index) in pagerank_scores.keys() else 0.0
        results.append(Counter({ 'TEXTRANK_SCORE': score }))
    return results

In [9]:
def textrank_keyword(text):
    txt = u' '.join([ su.text for su in text ])
    # Gets a dict of word -> lemma
    tokens = textcleaner.clean_text_by_word(text, 'english')
    split_text = list(textcleaner.tokenize_by_word(txt))

    # Creates the graph and adds the edges
    graph = commons.build_graph(keywords._get_words_for_graph(tokens))
    keywords._set_graph_edges(graph, tokens, split_text)
    del split_text # It's no longer used
    commons.remove_unreachable_nodes(graph)

    # # Ranks the tokens using the PageRank algorithm. Returns dict of lemma -> score
    pagerank_scores = keywords._pagerank_word(graph)
    extracted_lemmas = keywords._extract_tokens(graph.nodes(), pagerank_scores, 0.2, None)
    lemmas_to_word = keywords._lemmas_to_words(tokens)
    keyWords = keywords._get_keywords_with_score(extracted_lemmas, lemmas_to_word)
    # # text.split() to keep numbers and punctuation marks, so separeted concepts are not combined
    combined_keywords = keywords._get_combined_keywords(keyWords, txt.split())
    kw_scores = keywords._format_results(keyWords, combined_keywords, False, True)

    results = [ Counter({ 'TEXTRANK_KEYWORD_SCORE': keyword_mean_score(su.basic, kw_scores) }) for su in text ]
    return results

In [10]:
def keyword_mean_score(sentence, wordScores):
    totalScore = sum([ s for w, s in wordScores if w in sentence ])
    return totalScore / len(sentence.split())

In [11]:
def lexrank_keyphrase(text):
    results = []
    for i in range(len(text)):
        results.append(Counter({ 'LEXRANK_SCORE': 0.0 }))
    return results

In [12]:
def train_classifier(features, labels):
    print "STAGE [4] -- TRAINING MODEL -- Logistic Regression ..."
    print '<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>'
    vectorizer = DictVectorizer(sparse=False)
    feature_matrix = vectorizer.fit_transform(features) # Features = List of counters
    mod = LogisticRegression(fit_intercept=True, intercept_scaling=1, class_weight='auto')
    mod.fit_transform(feature_matrix, labels)
    return mod, feature_matrix, vectorizer

In [13]:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# TRAIN:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
features = featurize(documents, surfaceFeatures)
# del surfaceFeatures
model, featMatrix, vectorizer = train_classifier(features, labels)

scores = cross_val_score(model, featMatrix, labels, scoring="f1_macro") # accuracy, f1, log_loss
print model.coef_
predictions = model.predict(featMatrix)
print metrics.classification_report(labels, predictions)

STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ...
STAGE [4] -- TRAINING MODEL -- Logistic Regression ...
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
[[-0.06978593 -0.15334872 -0.25510274  0.0126926   0.1922343  -0.24831293
  -0.02436139  0.68096096  0.79596326 -0.22309355 -0.43292563  0.05515176
  -0.61671967  0.36719587 -0.24619082  0.022105    0.26400959 -0.03795395
   0.06248649 -0.43588031  0.1884946   0.60676931 -0.11005708 -0.28079814]]
             precision    recall  f1-score   support

       BODY       0.96      0.72      0.82       151
    SUMMARY       0.16      0.67      0.25        12

avg / total       0.90      0.71      0.78       163



In [14]:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# TEST:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
testXMLFolder = "sample_test"
testDocuments, testLabels = parse_xml_folder(testXMLFolder, summaryFolder, bodyFolder, writeOption='none')
testSurfaceFeatures = process_data(testDocuments)

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
STAGE [1] -- PARSING XML -- from sample_test ...
STAGE [2] -- PROCESSING DATA -- (tokenizing/tagging/stopwords/extracting) ...


In [15]:
testFeatures = featurize(testDocuments, testSurfaceFeatures)
testFeatureMatrix = vectorizer.transform(testFeatures) # Features = List of counters
del testSurfaceFeatures

STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ...


In [16]:
def evaluate_trained_classifier(model, feature_matrix, labels):
    """Evaluate model, the output of train_classifier, on the test data."""
    print "STAGE [5] -- TESTING -- Logistic Regression ..."
    print '<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>'
    predictions = model.predict(feature_matrix)
    print cross_val_score(model, feature_matrix, labels, scoring="f1_macro")
    return metrics.classification_report(labels, predictions)

In [17]:
print evaluate_trained_classifier(model, testFeatureMatrix, testLabels)

STAGE [5] -- TESTING -- Logistic Regression ...
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
[ 0.50747757  0.45185185  0.55555556]
             precision    recall  f1-score   support

       BODY       0.91      0.58      0.71       100
    SUMMARY       0.11      0.45      0.17        11

avg / total       0.83      0.57      0.65       111



In [18]:
def extract_summary(document):
    summary = []
    for su in document:
        if su.label == 'summary':
            summary.append(su.text)
        else:
            break
    return summary

In [21]:
from heapq import nlargest

def extract_top_ranked(document, featureMatrix, num):
    predictions = model.predict_proba(featureMatrix)
    itr = range(len(document))
    topIndexes = nlargest(num, itr, key=lambda i: predictions[i][1])
    topSentences = [ document[index].text for index in topIndexes]
    return topSentences

In [24]:
featIndex = 0
rouge_gold_summaries = []
rouge_generated_summaries = []
for i, tdoc in enumerate(testDocuments):
    numSentences = len(tdoc)
    rouge_gold_summaries.append(extract_summary(tdoc))
    rouge_generated_summaries.append(extract_top_ranked(tdoc, testFeatureMatrix[featIndex:featIndex+numSentences], 10))
    featIndex += numSentences

print rouge_gold_summaries
print rouge_generated_summaries

[[u'The Most Illustrious Order of Saint Patrick is a British order of chivalry associated with Ireland.', u'The Order was created in 1783 by George III.', u'The regular creation of knights of Saint Patrick lasted until 1921, when most of Ireland became independent as the Irish Free State.', u'While the Order technically still exists, no knight of St Patrick has been created since 1936, and the last surviving knight, Prince Henry, Duke of Gloucester, died in 1974.', u'The Queen, however, remains the Sovereign of the Order, and one officer, the Ulster King of Arms (now combined with Norroy King of Arms), also survives.', u'St Patrick is patron of the order; its motto is Quis separabit?, Latin for "Who will separate us?', u'": an allusion to the Vulgate translation of Romans 8:35, "Who shall separate us from the love of Christ?"', u'Most British orders of chivalry cover the entire kingdom, but the three most exalted ones each pertain to one constituent nation only.', u'The Order of St Pat